<a href="https://colab.research.google.com/github/vdobrovolskii/hypernyms/blob/master/Regularizing_Hypernym_Prediction_with_Recurrent_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regularizing Hypernym Prediction with Recurrent Networks

## Preparing the environment

In [0]:
!pip install gensim keras pandas

In [0]:
import re
import pickle
import random
import pickle
from collections import defaultdict
from xml.etree import ElementTree as ET
from math import ceil

import numpy as np
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle as shuffle_multiple_arrays

from keras.models import Model, load_model
from keras.layers import Dense, Input, PReLU
from keras.losses import cosine_proximity
import pandas as pd

In [0]:
pd.set_option('display.max_colwidth', 500)

Run the following cell if you are planning to use Google Colab

In [0]:
from google.colab import drive
drive.mount("/content/gdrive")

# Modify the variable for it to contain the directory where the files are located
path = "/content/gdrive/My Drive/Colab Notebooks/rhpwrn/"

## Helpers

In [0]:
def build_vocab(vectors):
    """
    Creating a dictionary of all the entries in the object holding the vectors.
    For instance, for vectors of "мама_NOUN#7", "мама_NOUN#9", "мама_NOUN#10" we get:
    "мама_NOUN" : ["7", "9", "10"]
    
    Parameters:
        vectors (gensim.models.KeyedVectors): sense embeddings object. Refer to
            https://radimrehurek.com/gensim/models/keyedvectors.html
        
    Returns:
        dict: see above
    """
    word_sense_pattern = re.compile('(.+)#(\d+)')
    vocab = defaultdict(lambda: [])
    for key in vectors.vocab.keys():
        match_obj = re.match(word_sense_pattern, key)
        vocab[match_obj.group(1)].append(match_obj.group(2))
    return dict(vocab)

In [0]:
def iter_wiktionary(filename, prefix):
    """
    Memory-friendly xml iteratation.
    
    Parameters:
        filename (str): the name of the file containing the xml dump of Wiktionary.
        prefix (str): the prefix used in each tag of the xml file.
        
    Yields:
        xml.etree.ElementTree.Element: element tagged "page"
    """
    context = iter(ET.iterparse(filename, events=("start", "end")))
    event, root = context.__next__()
    for event, elem in context:
        if event == "end" and elem.tag == prefix + "page":
            yield elem
            elem.clear()
            root.clear()

            
def parse_wiktionary(filename, vocab, verbose=False):
    """
    Creates a dictionary with all the hyponym-hypernym pairs found in Wiktionary
    for which there exist trained vectors.
    
    Parameters:
        filename (str): the name of the file containing the xml dump of Wiktionary.
        vocab (dict or set): an object containing all the words that we have vectors for.
        verbose (bool): whether or not to display progress.
        
    Returns:
        dict: a dictionary where every key is a vocab item and every value is a list of sets
            for example: "ключ_NOUN": [set(["инструмент_NOUN, приспособление_NOUN"]),
                                       set(["водоем_NOUN"]),
                                       set(["знак_NOUN", "символ_NOUN"])]
    
    """
    if verbose:
        print("\n===== Parsing Wiktionary =====")
        
    POS = {"сущ":"NOUN",
           "собств":"PROPN",
           "гл":"VERB",
           "прил":"ADJ",
           "числ":"NUM",
           "мест":"ADJ",
           "прич":"ADJ"}
    prefix = "{http://www.mediawiki.org/xml/export-0.10/}"
    section_pattern = re.compile(r"^=== Морфологические и синтаксические свойства ===\n(?:[^\n]*?\n)*?{{([а-я]+)"
                                 + r" ru.+?"
                                 + r"^==== Гиперонимы ====\n((?:#[^\n]*\n)+)"
                                 + r".+?"
                                 + r"^==== Гипонимы ====\n((?:#[^\n]*\n)+)", flags=re.M|re.S)
    hypernym_pattern = re.compile(r"\[\[([A-Za-zА-ЯЁа-яё-]+)\]\]")
    yo_pattern = re.compile(r"ё", flags=re.I)
     
    to_hypernyms = defaultdict(lambda: [])
    to_hyponyms = defaultdict(lambda: []) ###
    counter = 0
    
    for page in iter_wiktionary(filename, prefix):
        title = page.find(prefix + "title").text
        text = page.find(prefix + "revision").find(prefix + "text").text
        if text is None:
            continue
        matches = re.findall(section_pattern, text)
        if not matches:
            continue
        for match in matches:
            pos = match[0]
            if not pos in POS:
                continue
            if pos == 'сущ' and title[0].isupper():
                pos = 'собств'
            processed_title = re.sub(yo_pattern, 'е', title.lower()) + f"_{POS[pos]}"
            if not processed_title in vocab:
                    continue
            for hypernyms_group in match[1].splitlines():
                hypernyms = re.findall(hypernym_pattern, hypernyms_group)
                if not hypernyms:
                    continue                
                filtered_hypernyms = set()
                for hypernym in hypernyms:
                    hypernym = re.sub(yo_pattern, 'е', hypernym.lower()) + f"_{POS[pos if pos != 'собств' else 'сущ']}"
                    if hypernym in vocab:
                        filtered_hypernyms.add(hypernym)
                if not filtered_hypernyms:
                    continue
                to_hypernyms[processed_title].append(filtered_hypernyms)
            for hyponyms_group in match[2].splitlines():
                hyponyms = re.findall(hypernym_pattern, hyponyms_group)
                if not hyponyms:
                    continue                
                filtered_hyponyms = set()
                for hyponym in hyponyms:
                    hyponym = re.sub(yo_pattern, 'е', hyponym.lower()) + f"_{POS[pos if pos != 'собств' else 'сущ']}"
                    if hyponym in vocab:
                        filtered_hyponyms.add(hyponym)
                if not filtered_hyponyms:
                    continue
                to_hyponyms[processed_title].append(filtered_hyponyms)
                if verbose:
                    counter += 1
                    if counter % 100 == 0:
                        print(f"{counter}\n" if counter % 10000 == 0 else '.', end='', flush=True)
    if verbose:
        print(counter)
    return dict(to_hypernyms), dict(to_hyponyms)

In [0]:
class BasicQueue:
    """
    Non-complicated queue to perform BFS with.
    
    Methods:
        __init__(iterable)
        push(obj): add one object to queue
        extend(iterable): add multiple objects to queue
        __next__()
        __iter__()
    """
    def __init__(self, iterable):
        self._list = []
        self._i = 0
        self.extend(iterable)
        
    def push(self, obj):
        self._list.append(obj)
        
    def extend(self, iterable):
        self._list.extend(iterable)
        
    def __next__(self):
        if self._i >= len(self._list):
            raise StopIteration
        obj = self._list[self._i]
        self._i += 1
        if self._i == 2048:
            self._list = self._list[2048:]
            self._i = 0
        return obj
    
    def __iter__(self):
        return self

    
class VectorNet:
    """
    WordNet-like hierarchy where the nodes are sense vectors.
    
    Methods:
        __init__(to_hypernyms, to_hyponyms, vectors, vocab)
        build_training_data(test_size=0.025)
    """
    def __init__(self, to_hypers, to_hypos, vectors, vocab):
        self._vectors = vectors
        self._vocab = vocab
        self._up = defaultdict(lambda: set())
        self._down = defaultdict(lambda: set())
        for hyper, hyposets in to_hypos.items():
            for hyposet in hyposets:
                for hypo in hyposet:
                    self._up[hypo].add(hyper)
                    self._down[hyper].add(hypo)
        for hypo, hypersets in to_hypers.items():
            for hyperset in hypersets:
                for hyper in hyperset:
                    self._up[hypo].add(hyper)
                    self._down[hyper].add(hypo)
        self._up = dict(self._up)
        self._down = dict(self._down)
        self._remove_extraneous_links()
        self._vup = dict()
        self._vdown = defaultdict(lambda: set())
        self._vectorize_data()
        
    def _is_hyper(self, w1, w2):
        """
        Checks if w1 is a hypernym of w2.
        """
        i = 0
        processed = set()
        parents = BasicQueue([w2])
        for parent in parents:
            processed.add(parent)
            if not parent in self._up:
                continue
            if w1 in self._up[parent]:
                return True
            for new in self._up[parent]:
                if not new in processed:
                    parents.push(new)
        return False
        
    def _remove_extraneous_links(self):
        """
        Removes recursion in the network.
        """
        for word, hypers in self._up.items():          
            if word in hypers:
                hypers.remove(word)
                self._down[word].remove(word)
            if word in self._down:
                bugs = hypers.intersection(self._down[word])
                if bugs:
                    hypers.difference_update(bugs)
            for hyper in hypers:
                if self._is_hyper(word, hyper):
                    for hypo in self._down[word].copy():
                        if self._is_hyper(hypo, hyper):
                            self._down[word].remove(hypo)
                            self._up[hypo].remove(word)                           
            to_remove = set()
            for hyper1 in hypers:
                for hyper2 in hypers - set([hyper1]):
                    if self._is_hyper(hyper2, hyper1):
                        to_remove.add(hyper2)
            for hyper in to_remove:
                if hyper in self._down:
                        self._down[hyper].remove(word)
                hypers.remove(hyper)
                
    def _find_optimal_pair(self, first, second):
        best = (0, None, None)
        for elem1 in first:
            for elem2 in second:
                similarity = self._vectors.similarity(elem1, elem2)
                if similarity > best[0]:
                    best = (similarity, elem1, elem2)
        return best
                
    def _get_hypernym(self, sense, level=1):
        current = sense
        for _ in range(level):
            if current in self._vup:
                current = self._vup[current]
            else:
                current = None
                break
        return current
    
    def _get_senses(self, item):
        if isinstance(item, str):
            return [f"{item}#{sense}" for sense in self._vocab[item]]
        elif isinstance(item, set):
            return [f"{word}#{sense}" for word in item for sense in self._vocab[word]]
        else:
            raise ValueError("item must be of type set or str.")
                
    def _vectorize_data(self):
        for word, hypers in self._up.items():
            hypersenses = self._get_senses(hypers)
            for sense in self._get_senses(word):
                score, sense, hypersense = self._find_optimal_pair([sense], hypersenses)
                if score > 0.6:                  
                    self._vup[sense] = hypersense
                    self._vdown[hypersense].add(sense)
                    
    def build_training_data(self, max_depth=9, test_size=0.025):
        """
        Builds and returns the data to be used for training and evaluating a NN.
        
        Parameters:
            max_depth (int): maximum hypernymy depth (min 1).
            test_size (float): range (0;1), the amount of data used for tests.
            
        Returns:
            tuple:
                x_train (numpy.ndarray): vectors fed to the models, shape (n_samples, n_features).
                y_train (numpy.ndarray): expected vectors, shape (n_samples, n_features).
                levels_train (numpy.ndarray): hypernymy depth for each sample pair, shape (n_samples, 1).
                x_test_vec (numpy.ndarray): data used for tests, shape (n_samples, n_features).
                x_test_str (list): list of str containing vector labels of the test data.
                y_test_str (list): list of str containing expected vector labels.
        """
        if max_depth < 1:
            raise ValueError("max_depth must be 1 or greater")
        if test_size <= 0 or test_size >= 1:
            raise ValueError("test_size must be in range (0;1)")
        x_train = []
        y_train = []
        levels_train = []
        x_test_vec = []
        x_test_str = []
        y_test_str = []
        senses = list(self._vup.keys())
        random.shuffle(senses)
        for i, sense in enumerate(senses):
            if i < len(senses) * test_size:
                x_test_str.append(sense)
                x_test_vec.append(self._vectors[sense])
                y_test_str.append(set())
            current_level = 0
            while current_level < max_depth:
                current_level += 1
                hypernym = self._get_hypernym(sense, current_level)
                if hypernym:
                    if i < len(senses) * test_size:
                        y_test_str[i].add(hypernym)
                    else:
                        x_train.append(self._vectors[sense])
                        y_train.append(self._vectors[hypernym])
                        levels_train.append(current_level)
                else:
                    break
        x_train = np.stack(x_train, axis=0)
        y_train = np.stack(y_train, axis=0)
        levels_train = np.stack(levels_train, axis=0)
        x_test_vec = np.stack(x_test_vec, axis=0)
        return x_train, y_train, levels_train, x_test_vec, x_test_str, y_test_str

In [0]:
class LeveledModel:
    def __init__(self, x, y, levels, vectors):
        x, y, levels = shuffle_multiple_arrays(x, y, levels, random_state=12345)
        self._models = dict()
        self._data_train = dict()
        self._depth_fractions_train = dict()
        self._vectors = vectors
        print(f"The deepest level is {np.max(np.unique(levels))}")
        recurrent_unit = [Dense(1200), PReLU(), Dense(600), PReLU(), Dense(600), PReLU(), Dense(300)]
        for depth in np.unique(levels):            
            self._models[depth] = self._build_model(recurrent_unit, depth)
            mask_train = levels == depth            
            self._data_train[depth] = (x[mask_train], y[mask_train])
            self._depth_fractions_train[depth] = x[mask_train].shape[0] / x.shape[0]
            print(f"    Train {depth:>2}: {x[mask_train].shape[0]:>5} examples")
            
    def predict(self, x, confidence_threshold=0.95, verbose=False):
        total_models = len(list(self._models.keys())) # calculate it elsewhere and keep?
        raw_predictions = [None for _ in range(total_models)]
        predictions = []
        confidence_data = []
        all_predictions = []
        if verbose:
            print("Getting raw predictions ", end='')
        for i, model in self._models.items():
            if verbose:
                print('.', end='')
            raw_predictions[i - 1] = model.predict(x)
        print(" Done\n\nCalculating distances and choosing the most appropriate prediction...")
        for i in range(x.shape[0]):
            for model_id in range(total_models):
                hyper, confidence = self._vectors.most_similar(positive=[raw_predictions[model_id][i]], topn=1)[0]           
                if model_id == 0:
                    predictions.append(hyper)
                    confidence_data.append(confidence)
                    all_predictions.append([])
                elif confidence_data[i] < confidence_threshold and confidence > confidence_data[i]:
                    predictions[i] = hyper
                    confidence_data[i] = confidence
                all_predictions[i].append((model_id + 1, hyper, confidence))
            if verbose:
                print(f"{i + 1}\n" if (i + 1) % 100 == 0 else '.', end='')
        if verbose:
            print((i + 1) if (i + 1) % 100 != 0 else '')
        return predictions, confidence_data, all_predictions  
            
    def evaluate(self, all=False):
        if not all:
            print("Warning: evaluating only 1st level hypernyms.")
            return self._models[1].evaluate(*self._data_test[1])
            
    def fit(self, epochs=1, validation_split=0.05):
        for i in range(epochs):
            acc = 0
            loss = 0
            val_acc = 0
            val_loss = 0
            
            print(f"Epoch {i + 1:>4}: ", end='')
            for depth, model in self._models.items():    
                history = model.fit(*self._data_train[depth], validation_split=validation_split, epochs=1, verbose=0, batch_size=1024)
                acc += history.history['acc'][-1] * self._depth_fractions_train[depth]
                loss += history.history['loss'][-1] * self._depth_fractions_train[depth]
                val_acc += history.history['val_acc'][-1] * self._depth_fractions_train[depth]
                val_loss += history.history['val_loss'][-1] * self._depth_fractions_train[depth]
                print('.', end='')
            print(f" Loss: {loss:7.5f}, Accuracy: {acc:7.5f}, Val. loss: {val_loss:7.5f}, Val. accuracy: {val_acc:7.5f}")
            
            
    @staticmethod
    def _build_model(recurrent_unit, depth):
        inputs = Input(shape=(300,))
        x = inputs
        for _ in range(depth):
            for layer in recurrent_unit:
                x = layer(x)
        outputs = x
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer="rmsprop",
                     loss=lambda y_true, y_pred: weighted_cosine(y_true, y_pred, 2 ** depth),
                     metrics=["accuracy"])
        return model

In [0]:
def weighted_cosine(y_true, y_pred, weight):
    """
    Custom weight function to be used in keras model training.
    Weights the standard cosine proximity by a parameter.
    More specifically, takes the cosine distance (1 - cosine_proximity), weighs
    it and transforms back to cosine_proximity. Also note that the standard keras
    cosine proximity is multiplied by -1 to target for minimizing the function.
    """
    cosine_distance = 1 + cosine_proximity(y_true, y_pred)
    return -(1 - cosine_distance * weight)

In [0]:
def evaluate_model(x_str, y_true_str, y_pred_str, confidence_data, all_preds, vectors):
    """
    Outputs information on the model's performance.
    
    Parameters:
        x_str (list): labels of vectors fed to the model.
        y_true_str (list): labels of vectors that are expected from the model.
        y_pred_str (list): labels of vectors that the model has predicted.
        confidence_data (list): cosines between raw predictions and predicted labels.
        
            Note: lengths of all the previous parameters should be equal.
            
        vectors (gensim.models.KeyedVectors): sense embeddings object. Refer to
            https://radimrehurek.com/gensim/models/keyedvectors.html
        
    Returns:
        pandas.DataFrame: "auto_eval": are_same(standard, predicted)
                          "stimuli": label of sample x
                          "standard": label of sample y
                          "predicted": predicted label
                          "similarity": cosine(standard, predicted)
                          "confidence": cosine(raw_prediction, predicted)
    """
    rows = []
    for i in range(len(x_str)):
        stimulus = x_str[i]
        standard = y_true_str[i]
        predicted = y_pred_str[i]
        confidence = confidence_data[i]
        auto_eval = predicted in standard
        rows.append(pd.Series({"auto_eval":auto_eval,
                   "stimuli":stimulus,
                   "standard":", ".join(standard),
                   "predicted":predicted,
                   "all_preds":", ".join([f"({e[0]}, {e[1]}, {e[2]:1.5f})" for e in all_preds[i]]),
                   "confidence":confidence}))
        print('.' if auto_eval else 'x', end='\n' if (i + 1) % 100 == 0 else '')
    df = pd.DataFrame(rows)
    print("\n")
    print(f"Correct:         {df['auto_eval'].sum()} out of {len(df)}")
    print()
    print("Be careful when interpreting the results as they still require human evaluation.")
    return df

## Training

In [0]:
VECTOR_FILE = path + "sense_vectors.bin"
WIKI_FILE = path + "wiktionary.xml"
PARSED_WIKI = path + "parsed-wiki.pickle"
TRAINING_DATA = path + "data"

In [0]:
vectors = KeyedVectors.load_word2vec_format(VECTOR_FILE, binary=True, encoding='utf8', unicode_errors='ignore') 
vectors.init_sims(replace=True)

In [0]:
vocab = build_vocab(vectors)

Uncomment one of the following to either parse the Wiktionary database again or to use the pre-parsed data.

In [0]:
#to_hypers, to_hypos = parse_wiktionary(WIKI_FILE, vocab, verbose=True)

#with open(PARSED_WIKI, mode="rb") as f:
#    to_hypers, to_hypos = pickle.load(f)

In [0]:
vn = VectorNet(to_hypers, to_hypos, vectors, vocab)

Uncomment one of the following to either rebuild the training data or use the pre-built one.

In [0]:
#x_train, y_train, levels_train, x_test_vec, x_test_str, y_test_str = vn.build_training_data()

#with open(path + "data", mode="rb") as f:
#    x_train, y_train, levels_train, x_test_vec, x_test_str, y_test_str = pickle.load(f)

In [0]:
model = LeveledModel(x_train, y_train, levels_train, vectors)

Uncomment one of the following to either retrain the model or use the pre-trained one.

In [0]:
#model.fit(epochs=500)

#for i in range(1, 10):
#    model._models[i].load_weights(path + "model_weights" + str(i))

## Evaluation

In [0]:
y_pred_str, confidence_data, all_preds = model.predict(x_test_vec, confidence_threshold=0.97, verbose=True)

In [0]:
evaluate_model(x_test_str, y_test_str, y_pred_str, confidence_data, all_preds, vectors)